# uRPC

In [1]:
%discover
%connect test-argon mp

test-argon  mp://10.39.40.140:54321              
test-esp32  ws://10.39.40.135:8266               
test-esp32  serial:///dev/cu.usbserial-0160B5B8  
Connected to test-argon @ mp://10.39.40.140:54321


## Example Usage

The primary objective of the Airlift library is accesing networking features on the server. This is implemented with RPC calls from the client to the server.

Although presumably of limited usefulness, the uRPC library can also be used directly.

In [1]:
from urpc import *
print(version_())

V1


In [1]:
# import_(...) imports modules on the server.
# They can be used on the client as usual.

builtins_ = import_('builtins')
print(builtins_.sorted([4, 2, 7, 1]))

math_ = import_('math')
print(math_.log2(1024))

(1, 2, 4, 7)
10.0


In [ ]:
# use get_(...) to access attributes

sys_ = import_('sys')
import sys

print("server:", sys_.get_('platform'))
print("client:", sys.platform)

In [1]:
# execute arbitrary code on the server

exec_("import sys;  print('output from print forwarded to host: ', sys.platform)")
exec ("import sys;  print('print on host works, of course:      ', sys.platform)")

output from print forwarded to host:  esp32
print on host works, of course:       nRF52840


In [1]:
# Exceptions are delegated to the client
# args[-1] contains the traceback on the server

try:
    exec_("print('divide by 0', 5/0)")
except ZeroDivisionError as e:
    print("error message:", e.args[0])
    print(e.args[-1])

error message: divide by zero
Traceback (most recent call last):
  File "/lib/urpc_server/async_server.py", line 118, in async_serve
  File "/lib/urpc_server/async_server.py", line 37, in ex
  File "<string>", line 1, in <module>
ZeroDivisionError: divide by zero



In [1]:
try:
    exec_(
"""
def recurse(n):
    if n < 0:
        print("recurse", n, "divide by zero!")
        x = 5/0
    else:
        recurse(n-1)

recurse(3)
""")
except ZeroDivisionError as e:
    print("error message:", e.args[0])
    print(e.args[-1])

recurse -1 divide by zero!
error message: divide by zero
Traceback (most recent call last):
  File "/lib/urpc_server/async_server.py", line 118, in async_serve
  File "/lib/urpc_server/async_server.py", line 37, in ex
  File "<string>", line 9, in <module>
  File "<string>", line 7, in recurse
  File "<string>", line 7, in recurse
  File "<string>", line 7, in recurse
  File "<string>", line 7, in recurse
  File "<string>", line 5, in recurse
ZeroDivisionError: divide by zero



In [1]:
# object equality is maintained, except id

s1 = import_('sys')
s2 = import_('sys')

print(s1, int.from_bytes(s1.ext_type.data, 'big'))
print(s2, int.from_bytes(s2.ext_type.data, 'big'))

print("== ", s1 == s2)
print("eq_", eq_(s1, s2))
print("id_", id_(s1) == id_(s2))
print("BUT: id", id(s1) == id(s2))

<module 'sys'> 133
<module 'sys'> 134
==  True
eq_ True
id_ True
BUT: id False


## Limitations

As the examples show, features on the server (esp32) can be accessed from the client more or less transparently. Notable exceptions are the `import_` "function" (don't forget the trailing `_`!) and accessing attributes (`get_`).

For frequently used classes, simple wrappers on the client eliminate these differences for the user. See `socket.py` or `network.py` for examples.

A more profound limitation concerns object equality. Objects on the server are represented on the client by proxy objects. These have a small footprint, essentially just storing an identifier of the object on the server. 

In some situations it's possible to create several proxies that all point to the same object on the server. For example, calling `import_` for the same module repeately returns distinct proxy objects. Mostly this is not a problem: equality still works correctly, but `id` returns different values.

A more subtle example involves `select.poll()`. The poller returns registered streams. The client each time returns a new proxy. As it happens, the current implementation of `uasyncio` relies on the id's of all sockets being identical. A "workaround" is replacing `id` in `uasyncio.core` with `id_`. A better solution is to run the poller natively on the client (TODO).

## Object Lifetime Management

When the gc on the client collects a proxy object, the finalizer notifies the server which removes the reference to the actual object corresponding to the proxy. When no references are left the server's gc collects the actual object. 

Since MicroPython does not support finalizers for user defined classes, the implementation relies on a `finalizerproxy` module (a small class written in C).

## Requirements

URPC uses the following features that are not part of "standard" Micro/CircuitPython:

* `msgpack`: merged into CircuitPython, but not yet MicroPython
* `finalizerproxy`: urpc works without, but then objects on the server will not be automatically recycled. Workaround: explicitly call `__del__()` on the proxy object.